In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Load the LLM model
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# Create prompt template
prompt_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "tell me about {topic}")
    ]
)

In [10]:
# create a chain consisting of a sequence of calls
chain = prompt_template | llm | StrOutputParser()

result = chain.invoke({'topic': 'the Netherlands'})
result

"The Netherlands, also known as Holland, is a country located in Northwestern Europe. It is known for its flat landscape, picturesque windmills, tulip fields, and extensive canal systems. The capital city is Amsterdam, which is famous for its historic architecture, vibrant cultural scene, and picturesque canals.\n\nThe Netherlands is a constitutional monarchy with a parliamentary system. It is a highly developed country with a strong economy, known for its innovative industries such as agriculture, technology, and design. The Dutch are also known for their liberal social policies, including a tolerant attitude towards drugs and a strong emphasis on environmental sustainability.\n\nThe Netherlands is a popular tourist destination, attracting visitors with its charming cities, beautiful countryside, and rich cultural heritage. Some of the country's other notable cities include Rotterdam, The Hague, and Utrecht. The Dutch are also known for their love of cycling, with extensive bike paths

## Under the hood

As we see above:
* we can define the different parts of a chain where each part performs a specific task,
* then, the operator `|` connects them.

Under the hood, 
* each part is defined as a `RunnableLambda` where it accept an input and generate an output, similar to `lambda` function in python.
* then different parts are connected by `RunnableSequence`.



In [4]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

In [7]:
prompt_format = RunnableLambda(lambda x: prompt_template.format_prompt(**x)) # note that the input for this function is a dictionary
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages())) # to_messages(): returns a list of messages
output = RunnableLambda(lambda x: x.content)

chain1 = RunnableSequence(
    first=prompt_format, # first component
    middle=[invoke_model], 
    last=output # last component
)
chain1.invoke({'topic': 'the Netherlands'})

"The Netherlands, also known as Holland, is a country located in Northwestern Europe. It is known for its flat landscape, picturesque windmills, tulip fields, and extensive canal systems. The capital city is Amsterdam, which is famous for its historic architecture, vibrant cultural scene, and picturesque canals.\n\nThe Netherlands is a constitutional monarchy with a parliamentary system. It is a highly developed country with a strong economy, known for its innovative industries, such as agriculture, technology, and design. The Dutch are also known for their progressive social policies, including a strong emphasis on environmental sustainability and social welfare.\n\nThe Netherlands is a popular tourist destination, attracting visitors with its charming cities, beautiful countryside, and rich cultural heritage. Some of the country's other notable cities include Rotterdam, The Hague, and Utrecht. The Dutch are also known for their love of cycling, with extensive bike paths and a bike-fr

In [10]:
chain2 = prompt_format | invoke_model | output
chain2.invoke({'topic': 'Netherlands'})

"The Netherlands, also known as Holland, is a country located in Northwestern Europe. It is known for its flat landscape, picturesque windmills, tulip fields, and extensive canal systems. The capital city is Amsterdam, which is famous for its historic architecture, vibrant cultural scene, and picturesque canals.\n\nThe Netherlands is a constitutional monarchy with a parliamentary system. It is a highly developed country with a strong economy, known for its innovative industries, such as agriculture, technology, and design. The Dutch are also known for their progressive social policies, including a strong emphasis on environmental sustainability and social welfare.\n\nThe Netherlands is a popular tourist destination, attracting visitors with its charming cities, beautiful countryside, and rich cultural heritage. Some of the country's other notable cities include Rotterdam, The Hague, and Utrecht. The Dutch are also known for their love of cycling, with extensive bike paths and a bike-fr

For more information about Runnables please check [Runnable](https://python.langchain.com/docs/how_to/lcel_cheatsheet/)